In [12]:
import nltk
import spacy
import pandas as pd
from tqdm import tqdm
import os
import nltk
import numpy as np
from pattern.text.en import singularize
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import pickle
from collections import Counter

In [13]:
#get all entities
ent_df = pd.read_csv("Litcovid_treatment_ents.csv")

#create multiindex df to sort by pmid
tuples = list(zip(*[ent_df['pmid'], ent_df['Unnamed: 0']]))
index = pd.MultiIndex.from_tuples(tuples, names=['pmid', 'orig_index'])
pmid_ent_df = ent_df.copy()
pmid_ent_df.drop(["Unnamed: 0"], axis=1, inplace=True)
pmid_ent_df.index = index

In [14]:
pmid_ent_df.head(10)

text         type  negation  \
pmid     orig_index                                                
31981224 0                       covid-19  Participant         0   
         1                       relevant  Participant         0   
         2           host defense against      Outcome         0   
         3                       covid-19  Participant         0   
31996494 4                       covid-19  Participant         0   
32013309 5                   inflammation      Outcome         0   
         6                       covid-19  Participant         0   
         7                      inhibited     modifier         0   
32015560 8                     accelerate     modifier         0   
         9                        control      Outcome         0   

                                                              sentence  \
pmid     orig_index                                                      
31981224 0           In this review , we provide an update on CoV i...   
         1           In this review , we provide an update on CoV i...   
         2           In this review , we provide an update on CoV i...   
         3           This highlights the importance of immune respo...   
31996494 4           As of January 22 , 2020 , a total of 571 cases...   
32013309 5           Here , we propose for the first time that infl...   
         6           Here , we propose for the first time that infl...   
         7           Here , we propose for the first time that infl...   
32015560 8                                    accelerate its control .   
         9                                    accelerate its control .   

                         pmid  
pmid     orig_index            
31981224 0           31981224  
         1           31981224  
         2           31981224  
         3           31981224  
31996494 4           31996494  
32013309 5           32013309  
         6           32013309  
         7           32013309  
32015560 8           32015560  
         9           32015560

In [47]:
#get all pmids with hcq
sev_pmids = ent_df[(ent_df.text == "severe") & (ent_df.type == "Outcome")].pmid.unique()

#get entities with hcq pmids
sev_ent_df = pmid_ent_df.loc[sev_pmids]
severe_pmids_df = sev_ent_df[(sev_ent_df.text == "severe")&(sev_ent_df.type == "Outcome")]

In [48]:
severe_pmids_df.drop(["type", "negation", "pmid"], axis=1, inplace=True)

C:\Users\janep\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [49]:
severe_pmids_df["subtag"] = ""

C:\Users\janep\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [51]:
severe_pmids_df.reset_index(inplace=True)
severe_pmids_df.drop(["orig_index"], axis=1, inplace=True)

C:\Users\janep\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [53]:
severe_pmids_df.drop("index", axis=1, inplace=True)

C:\Users\janep\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [54]:
severe_pmids_df

,pmid,text,sentence,subtag
0,32052373,severe,This paper summarizes important considerations...,
1,32061198,severe,The novel coronavirus 2019 ( COVID-19 ) infect...,
2,32061198,severe,Increased ACE and Ang II are poor prognostic f...,
3,32077660,severe,"ALT , AST , ALB and TBil of the severe or crit...",
4,32098422,severe,severe symptoms in respiratory tract infections .,
5,32145714,severe,Result : s ( 1 ) Of the 16 pregnant women with...,
6,32164090,severe,multiple mottling and ground-glass opacity.Com...,
7,32164092,severe,The novel coronavirus 2019 ( COVID-19 ) infect...,
8,32164092,severe,Increased ACE and Ang II are poor prognostic f...,
9,32164424,severe,The reported severe/critical case ratio is app...,


In [63]:
jane_pmids = int(round(severe_pmids_df.shape[0])/30)

In [67]:
severe_pmids_df.iloc[:jane_pmids].to_csv("jane_severe_pmids.csv")

In [68]:
severe_pmids_df.iloc[jane_pmids:jane_pmids*2].to_csv("hao_severe_pmids.csv")

In [69]:
severe_pmids_df.iloc[jane_pmids*2:jane_pmids*2*3].to_csv("fengyang_severe_pmids.csv")